# Test bug on RAGxplorer on query
Without brackets around the query in the openai client with chroma, embeddings are created for each word, not each string.

In [2]:
import os
from ragxplorer import RAGxplorer

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)
import json
import chromadb
import pandas as pd

KeyboardInterrupt: 

In [ ]:
query="What are examples of lubricants which should be avoided for space mechanism applications?"

OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
embedding_model='text-embedding-ada-002'

data_path='../data/AMS/'
pdf='AMS_2022.pdf'

index_path='../db/chromadb/'
index_name='chromadb-openai-ams'

viz_data='../data/AMS/ams_data-400-0-50.json'

In [ ]:
with open(viz_data, 'r') as f:
    data = json.load(f)

index_name=data['visualization_index_name']
umap_params=data['umap_params']
viz_data=pd.read_json(data['viz_data'], orient='split')

In [ ]:
rx_client = RAGxplorer(embedding_model=embedding_model)
chroma_client = chromadb.PersistentClient(path='../db'+'/chromadb/')
collection=chroma_client.get_collection(name=index_name,embedding_function=rx_client._chosen_embedding_model)

In [ ]:
rx_client._query.original_query = query

In [ ]:
rx_client.load_chroma(collection,
                     umap_params=umap_params,
                     initialize_projector=True)

In [ ]:
# All of these do the same thing, tested

query_embedding_chroma=rx_client._chosen_embedding_model([rx_client._query.original_query])
# query_embedding=rx_client._vectordb._embedding_function(rx_client._query.original_query)
# query_embedding=embedding_function(query)

In [ ]:
len(rx_client._query.original_query)

In [ ]:
fig = rx_client.visualize_query(query,
                                import_projection_data=viz_data,
                                verbose=True)

In [ ]:
from openai import OpenAI
client = OpenAI()

# Done correctly, see here: https://platform.openai.com/docs/guides/embeddings/use-cases
openai_embedding=client.embeddings.create(input = [query], model=embedding_model).data[0].embedding

# Endpoints

In [ ]:
import json

config_file = '../config/config.json'
with open(config_file, 'r') as f:
        config = json.load(f)
        databases = {db['name']: db for db in config['databases']}
        llms  = {m['name']: m for m in config['llms']}

In [ ]:
hf_models=llms['Hugging Face']['models']
model_names= [item['model'] for item in hf_models]
model_endpoints= [item['endpoint'] for item in hf_models]


In [ ]:
model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
endpoint = next(item['endpoint'] for item in hf_models if item['model'] == model_name)

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
# from langchain_community.llms import HuggingFaceTextGenInference
from langchain_community.llms import HuggingFaceHub
import os

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

api_key=os.getenv('HUGGINGFACEHUB_API_TOKEN')

# llm = ChatOpenAI(
#                     model_name="tgi",
#                     openai_api_key=api_key,
#                     openai_api_base=endpoint + "/v1/",
#                 )




# llm = HuggingFaceTextGenInference(
#     inference_server_url=endpoint,
#     # max_new_tokens=512,
#     # top_k=10,
#     # top_p=0.95,
#     # typical_p=0.95,
#     temperature=0.01,
#     # repetition_penalty=1.03,
# )
# llm("What did foo say about bar?")

# model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
# model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# model_name = "meta-llama/Llama-2-70b-chat-hf"

llm = HuggingFaceHub(repo_id=model_name,
                     model_kwargs={"temperature": 0.1, "max_length": 516})


In [ ]:
prompt="""
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
---
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Include sources from the chat history in the standalone question created.
---

Chat History:

User Question: What are examples of lubricants which should be avoided for space mechanism applications?
Standalone Question:
"""

response=llm.predict(prompt)
print(response)

# Advanced RAG

In [117]:
import os
import chromadb

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as lancghain_Document
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Pinecone

from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel
from langchain.prompts.prompt import PromptTemplate

from pinecone import Pinecone as pinecone_client

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

local_db_path='../db/'
chunk_size=500
chunk_overlap=0
embedding_model='text-embedding-ada-002'

llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'), model_name="gpt-3.5-turbo", temperature=0.1)
query_model=OpenAIEmbeddings(model=embedding_model,openai_api_key=os.getenv('OPENAI_API_KEY'))
store=InMemoryStore()


In [118]:
# index_type='chroma'
index_type='pinecone'

type='standard'
# type='parent-child'

if index_type=='chroma':
    # Chroma
    persistent_client = chromadb.PersistentClient(path=local_db_path+'/chromadb')   
    vectorstore = Chroma(client=persistent_client,
                        collection_name='parent-child-test',
                        embedding_function=query_model)
elif index_type=='pinecone':
    # Pinecone
    pc=pinecone_client(api_key=os.getenv('PINECONE_API_KEY'))
    if type=='standard':
        index=pc.Index('pinecone-openai-ams')
    elif type=='parent-child':
        index=pc.Index('pinecone-openai-ams-pc')
    vectorstore = Pinecone(index, query_model, "page_content")

In [119]:
# persistent_client.delete_collection(name="chromadb-openai")

In [120]:
docs=['../data/AMS/AMS_2006.pdf','../data/AMS/AMS_2012.pdf']

In [121]:
docs_out=[]
for doc in docs:
    loader = PyPDFLoader(doc)
    data = loader.load()
    # pages=text_splitter.split_documents(data)
    # for page in pages:
    #     # Add metadata to the end of the page content, some RAG models don't have metadata.
    #     page.page_content += str(page.metadata)
    #     doc_temp=lancghain_Document(page_content=page.page_content,
    #                                 source=page.metadata['source'],
    #                                 page=page.metadata['page'],
    #                                 metadata=page.metadata)
    docs_out.extend(data)

In [135]:
data[101]

Document(page_content='88 between the locked and unlocked condition of the TO Isolators  during the landing impacts and assess LS-\nDYNA® modeling techniques and predictive capability. The test configuration had a crew pallet-mockup \n(green, Figure 7) suspended from a cage (yellow, Figure 7) via the eight struts simulating the Orion CM interior. The cage was housed in a ring base/support structure to provide adjustment for various landing parachute pitch angles. Vertical drop tests were conducted by releasing the test fixture from a crane hook at a height calculated to produce the desired impact velocity. Tapered stacks of paper honeycomb at the four corners of the test fixture base were used to produce impact pulses approximating Orion water landings (see Figure 7). Measurements recorded during the tests include TO Isolator displacements, forces and accelerations as well as pallet and cage accelerations. High-speed cameras and photogrammetry were used to verify impact conditions and 

In [122]:
questions=["What are some challenges associated with light cover mechanisms?"]

In [123]:
from langchain import hub
from langchain.prompts.prompt import PromptTemplate

# Prompts on the hub: https://smith.langchain.com/hub/my-prompts?organizationId=45eb8917-7353-4296-978d-bb461fc45c65
CONDENSE_QUESTION_PROMPT = hub.pull("dmueller/ams-chatbot-qa-condense-history")
QA_PROMPT=hub.pull("dmueller/ams-chatbot-qa-retrieval")
QA_WSOURCES_PROMPT=hub.pull("dmueller/ams-chatbot-qa-retrieval-wsources")
QA_GENERATE_PROMPT=hub.pull("dmueller/generate_qa_prompt")
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

## Standard RAG

In [124]:
# Combine documents, from queries.py

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [125]:
# From queries.py

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [129]:
retriever=vectorstore.as_retriever()

In [130]:
# Directly from queries.py
# This adds a 'memory' key to the input object
loaded_memory = RunnablePassthrough.assign(
                    chat_history=RunnableLambda(memory.load_memory_variables) 
                    | itemgetter('history'))  

# Assemble main chain
standalone_question = {
    'standalone_question': {
        'question': lambda x: x['question'],
        'chat_history': lambda x: get_buffer_string(x['chat_history'])}
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser()}

retrieved_documents = {
    'source_documents': itemgetter('standalone_question') 
                        | retriever,
    'question': lambda x: x['standalone_question']}

# Now we construct the inputs for the final prompt
final_inputs = {
    'context': lambda x: _combine_documents(x['source_documents']),
    'question': itemgetter('question')}

# And finally, we do the part that returns the answers
answer = {
    'answer': final_inputs 
                | QA_PROMPT 
                | llm,
    'references': itemgetter('source_documents')}
conversational_qa_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [127]:
inputs = {"question": questions[0]}
result = conversational_qa_chain.invoke(inputs)
result

{'answer': AIMessage(content='Some challenges associated with light cover mechanisms include maintaining durability while keeping the cover lightweight, ensuring proper insulation and protection from external elements, and finding materials that are both transparent and resistant to wear and tear.'),
 'references': [Document(page_content="self-deploying, roll-up, film-type coverasthebestapproach tomeettheMIPsystem requirements. The roll-upcoverprovided several distinct benefits. •Itrequired veryfewmechanical elements tooperate. Thissaveddesigntime,development time,and reduced mass. •Therollingactionofthecovercouldcapture dustasitrolledup,minimizing disturbances •Thefactthatthecoverrolleditselfupmeantthatthecoverwouldrequire minimal envelope after deployment. •Theminimization ofmechanisms anddrivenelements reduced thepowerandcomplexity ofcontrol electronics required foroperation. •Theshapeandsizeofthecovermadeitextremely adaptable tothecomplex shapeandfunctionality required •Although so

In [128]:
result['references']

[Document(page_content="self-deploying, roll-up, film-type coverasthebestapproach tomeettheMIPsystem requirements. The roll-upcoverprovided several distinct benefits. •Itrequired veryfewmechanical elements tooperate. Thissaveddesigntime,development time,and reduced mass. •Therollingactionofthecovercouldcapture dustasitrolledup,minimizing disturbances •Thefactthatthecoverrolleditselfupmeantthatthecoverwouldrequire minimal envelope after deployment. •Theminimization ofmechanisms anddrivenelements reduced thepowerandcomplexity ofcontrol electronics required foroperation. •Theshapeandsizeofthecovermadeitextremely adaptable tothecomplex shapeandfunctionality required •Although somedevelopment wasnecessary thesimple design allowed forveryrapiddesign and development tomeetthecompressed schedule ofalastminutecoverprogram. 74{'source': 'AMS_2001.pdf', 'page': 84}", metadata={'page': 84.0, 'source': 'AMS_2001.pdf'}),
 Document(page_content='Cover Development Although JPLhasdeveloped anumber ofde

## Parent-child
https://colab.research.google.com/github/datastax/ragstack-ai/blob/main/examples/notebooks/advancedRAG.ipynb

In [113]:
# index_type='chroma'
index_type='pinecone'

# type='standard'
type='parent-child'

if index_type=='chroma':
    # Chroma
    persistent_client = chromadb.PersistentClient(path=local_db_path+'/chromadb')   
    vectorstore = Chroma(client=persistent_client,
                        collection_name='parent-child-test',
                        embedding_function=query_model)
elif index_type=='pinecone':
    # Pinecone
    pc=pinecone_client(api_key=os.getenv('PINECONE_API_KEY'))
    if type=='standard':
        index=pc.Index('pinecone-openai-ams')
    elif type=='parent-child':
        index=pc.Index('pinecone-openai-ams-pc')
    vectorstore = Pinecone(index, query_model, "page_content")

In [115]:
k_parent=5
parent_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size*k_parent, chunk_overlap=chunk_overlap)
child_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# Create a parent document retriever
parent_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [116]:
# Split and load the documents into the vector and parent stores
parent_retriever.add_documents(docs_out)

In [131]:
retriever=parent_retriever

# Directly from queries.py
# This adds a 'memory' key to the input object
loaded_memory = RunnablePassthrough.assign(
                    chat_history=RunnableLambda(memory.load_memory_variables) 
                    | itemgetter('history'))  

# Assemble main chain
standalone_question = {
    'standalone_question': {
        'question': lambda x: x['question'],
        'chat_history': lambda x: get_buffer_string(x['chat_history'])}
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser()}

retrieved_documents = {
    'source_documents': itemgetter('standalone_question') 
                        | retriever,
    'question': lambda x: x['standalone_question']}

# Now we construct the inputs for the final prompt
final_inputs = {
    'context': lambda x: _combine_documents(x['source_documents']),
    'question': itemgetter('question')}

# And finally, we do the part that returns the answers
answer = {
    'answer': final_inputs 
                | QA_PROMPT 
                | llm,
    'references': itemgetter('source_documents')}
conversational_qa_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [132]:
inputs = {"question": questions[0]}
result = conversational_qa_chain.invoke(inputs)
result

{'answer': AIMessage(content='Some challenges associated with light cover mechanisms include maintaining durability while keeping the cover lightweight, ensuring proper insulation and protection from external elements, and designing mechanisms that are easy to operate and maintain.'),
 'references': [Document(page_content='JWST NlRSpec Cryogenic Light Shield Mechanism \nKathleen Hale’ and Rajeev Sharma’ \nAbstract \nThe focal plane detectors for the Near-Infrared Spectrometer (NIRSpec) instrument on the James Webb \nSpace Telescope (JWST) require a light tight cover for calibration along with an open field-of-view during \nground performance testing within a cryogenic dewar. In order to meet the light attenuation requirements \nand provide open and closed fields of view without breaking vacuum, a light shield mechanism was \ndesigned. This paper describes the details of the light shield mechanism design and test results. Included \nis information on the labyrinth light path design, mot

# Module testing

In [1]:
import sys
# Assuming `/home/user/projects/my_modules` is the path to the folder with your modules
sys.path.append('../src')

In [4]:
import data_processing
import glob

In [ ]:
data_folder = '../data/AMS/'
docs = glob.glob(data_folder+'*.pdf')   # Only get the PDFs in the directory

json_file = data_folder+'ams_data-400-0-0.json'
chunk_size = 500
chunk_overlap = 0
use_json = False
show_progress = True

data_processing.chunk_docs(docs,
                        file=json_file,
                        chunk_size=chunk_size,
                        chunk_overlap=chunk_overlap,
                        use_json=use_json,
                        show_progress=show_progress)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
chunk_size=500
chunk_overlap=0

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=chunk_overlap)